# Trying the model


In [2]:
%cd ..

/workspace


In [5]:
# load
import torch
from open_gns.models import EncodeProcessDecode
from open_gns.dataset import GNSDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load data sample (1 frame)
dataset = GNSDataset('./notebooks', split='test')
input_size = dataset.num_node_features
print(input_size)
# Load model
model = EncodeProcessDecode(input_size).to(device)

# Perform rollout using the model
data = dataset[0]
data = data.to(device)

# Display every frame to visually verify


20
